In [1]:
# !git clone https://github.com/StarkWizard/cairo-llm.git

In [2]:
!cd /content/cairo-llm

In [3]:
!pip install -r /content/cairo-llm/requirements-Mac.txt

DEPRECATION: git+https://github.com/victorlee0505/ctransformers.git@vlee/transformers#egg=ctransformers[cuda] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-3m076kp_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-3m076kp_
  Resolved https://github.com/huggingface/trl.git to commit 685209716915b010098b7e18b05fbad42af864d3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/victorlee0505/ctransformers.git (to revision vlee/transformers) to /tmp/pip-install-gwj95idg/ctransformers_23960c120c284e89841cb09c1cea7c74
  Running command git clone --filter=

In [4]:
# Use %cd NOT !cd
%cd /content/cairo-llm/wizardlib
!pip install -e .

/content/cairo-llm/wizardlib
Obtaining file:///content/cairo-llm/wizardlib
  Preparing metadata (setup.py) ... done
  Attempting uninstall: wizardlib
    Found existing installation: wizardlib 0.1
    Uninstalling wizardlib-0.1:
      Successfully uninstalled wizardlib-0.1
  Running setup.py develop for wizardlib


In [5]:
import torch
import wizardlib as wizard
from rich.console import Console
import argparse
console = Console()

In [6]:
# Remove Annoying Warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="transformers.utils.generic")
warnings.filterwarnings("ignore", category=UserWarning, module="trl.trainer.ppo_config")
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly")

In [7]:
ok_prefix = "[bold green]->[/bold green]"
nok_prefix = "[bold red]->[/bold red]"

platform = wizard.getPlatform()
hasGPU = wizard.hasGPU(platform)

In [20]:
# Test GPU Availability
if not hasGPU:
    console.print(nok_prefix + "[bold red]GPU is not available. Please make sure your system has a compatible GPU.[/bold red]")
    console.print("You either need a PC with an NVdia GPU or a Mac with Apple M1/M2/M3 GPU.")
    exit(1)

gpuDevice = wizard.getDevice(platform)

console.print(f"Platform Detected: [bold green]{platform}[/bold green] with device [bold green]{gpuDevice}[/bold green]")

->GPU is not available. Please make sure your system has a compatible GPU.

You either need a PC with an NVdia GPU or a Mac with Apple M1/M2/M3 GPU.

Platform Detected: <module 'platform' from '/usr/lib/python3.10/platform.py'> with device cuda

In [9]:
import platform
print(platform.platform())

Linux-6.1.58+-x86_64-with-glibc2.35


In [10]:
from huggingface_hub import login
login()

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [12]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
# import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [13]:
filename = "/content/LabeledText.xlsx"
print("File Path:", filename)

column_indices = [1,2]

df = pd.read_excel(filename)
print(df.shape)

df = df.rename(columns={'Caption' : 'text', 'LABEL' : 'sentiment'})

print(df['sentiment'].unique())

print(df.isnull().sum())

X_train = list()
X_test = list()
for sentiment in ["positive", "neutral", "negative"]:
    train, test  = train_test_split(df[df.sentiment == sentiment],
                                    train_size=300,
                                    test_size=300,
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label 4, 3, 2, 1, 0.

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label 4, 3, 2, 1, or 0.

            [{data_point["text"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1),
                      columns=["text"])

y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

File Path: /content/LabeledText.xlsx
(4869, 3)
['negative' 'positive' 'neutral']
File Name    0
text         0
sentiment    0
dtype: int64


In [14]:
# params & hyperparams
default_epochs = 3
default_lora_r = 64
default_lora_alpha=16
default_lora_dropout=0.1
default_batch_size = 2
default_accumulation_steps = 1
default_lr = 2e-4
default_packing = False
default_window = 512
# models
# defaul_model_name = "codellama/CodeLlama-7b-Instruct-hf"
# default_tokenizer_name = "codellama/CodeLlama-7b-Instruct-hf"
# defaul_dataset_name = "StarkWizard/cairo-instruct"
# defaul_new_model = "StarkWizard/llama-2-7b-cairo-trained-PEFT"
default_max_seq_length = 1024

In [15]:
model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
                pretrained_model_name_or_path=model_name,
                trust_remote_code=True,
                torch_dtype=torch.float16,
                # low_cpu_mem_usage=True,
                device_map="auto"
                #  **additional_params,
            )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# model.resize_token_embeddings(len(tokenizer))
# model.config.use_cache=False
# model.config.pretraining_tp=1
# model.config.window = 4096
# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)
# model.to('cuda')

In [17]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [18]:
# Make sure to set temperature to a positive float (not 0)!
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.001,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [19]:
y_pred = predict(test, model, tokenizer)


  0%|          | 0/900 [00:00<?, ?it/s]


RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig ,prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        inference_mode = False
    )

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="ManuThakur/Llama2Trained",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    gradient_checkpointing = True,
    evaluation_strategy="steps",
    learning_rate=2e-4,
    lr_scheduler_type="constant",
    warmup_ratio=0.03,
    max_grad_norm=0.3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    num_train_epochs=3,
    group_by_length=True,
    fp16=False,
    # report_to=wand_to,
    push_to_hub=True,
    adam_beta2=0.999,
    do_train=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()